In [1]:
pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import time
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('API_KEY')
SEARCH_URL = "https://www.googleapis.com/youtube/v3/search"
VIDEOS_URL = "https://www.googleapis.com/youtube/v3/videos"

REGION_CODE = "FR"
MAX_PER_PAGE = 50      # max autorisé
TARGET_VIDEOS = 1000   # par niche


In [3]:
def collect_niche_videos(keywords, target=1000):
    video_ids = []

    for keyword in keywords:
        page_token = None

        while len(video_ids) < target:
            params = {
                "part": "snippet",
                "q": keyword,
                "type": "video",
                "order": "date",
                "maxResults": MAX_PER_PAGE,
                "key": API_KEY
            }

            if page_token:
                params["pageToken"] = page_token

            response = requests.get(SEARCH_URL, params=params)
            data = response.json()

            for item in data.get("items", []):
                video_ids.append(item["id"]["videoId"])

            page_token = data.get("nextPageToken")
            if not page_token:
                break

    # 🔑 déduplication
    video_ids = list(set(video_ids))

    return video_ids[:target]


In [4]:
# TEST SIMPLE : est-ce que search renvoie des résultats ?
test_ids = collect_niche_videos("fitness", target=10)
print(len(test_ids))
test_ids[:5]


0


[]

In [5]:
def fetch_video_details(video_ids):
    rows = []

    for i in range(0, len(video_ids), 50):
        batch = video_ids[i:i+50]

        params = {
            "part": "snippet,statistics,contentDetails",
            "id": ",".join(batch),
            "key": API_KEY
        }

        response = requests.get(VIDEOS_URL, params=params)
        data = response.json()

        for video in data.get("items", []):
            rows.append({
                "video_id": video["id"],
                "title": video["snippet"]["title"],
                "description": video["snippet"].get("description", ""),
                "channel": video["snippet"]["channelTitle"],
                "published_at": video["snippet"]["publishedAt"],
                "duration": video.get("contentDetails", {}).get("duration", None),
                "views": int(video["statistics"].get("viewCount", 0)),
                "likes": int(video["statistics"].get("likeCount", 0)),
                "comments": int(video["statistics"].get("commentCount", 0)),
                'channel_id': video['snippet'].get('channelId'),
                'category_id': video['snippet'].get('categoryId'),
                'language': video['snippet'].get('defaultAudioLanguage', 'N/A')
            })

        time.sleep(0.2)

    return pd.DataFrame(rows)


In [6]:
niches = {
    "lifestyle": ["minimalisme", "maison propre", "organisation maison", "décoration", "aesthetic","luxury lifestyle", "désencombrement", "méthode japonaise", "routine matinale", "old money", "intérieur luxueux"],
    "mode_beaute": ["tuto mode", "look", "chic parisien", "morphologie", "routine beauté", "maquillage", "coiffeur", "mode éco responsable", "seconde main"]
}

for niche, keywords in niches.items():
    print(f"Collecte : {niche}")
    ids = collect_niche_videos(keywords, TARGET_VIDEOS)
    df = fetch_video_details(ids)
    df.to_csv(f"market_{niche.replace(' ', '_')}_fr.csv", index=False)
    print(f"market_{niche.replace(' ', '_')}_fr.csv sauvegardé ({len(df)} vidéos)")


Collecte : lifestyle
market_lifestyle_fr.csv sauvegardé (0 vidéos)
Collecte : mode_beaute
market_mode_beaute_fr.csv sauvegardé (0 vidéos)


In [7]:
SEARCH_URL = "https://www.googleapis.com/youtube/v3/search"

params = {
    "part": "snippet",
    "q": "fitness",
    "type": "video",
    "maxResults": 5,
    "key": API_KEY
}

r = requests.get(SEARCH_URL, params=params)
print(r.status_code)
print(r.text)


403
{
  "error": {
    "code": 403,
    "message": "Method doesn't allow unregistered callers (callers without established identity). Please use API Key or other form of API consumer identity to call this API.",
    "errors": [
      {
        "message": "Method doesn't allow unregistered callers (callers without established identity). Please use API Key or other form of API consumer identity to call this API.",
        "domain": "global",
        "reason": "forbidden"
      }
    ],
    "status": "PERMISSION_DENIED"
  }
}

